<a href="https://colab.research.google.com/github/amar-naik/EAP/blob/master/Week5/Assignment_5_v2_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# mount gdrive and unzip data
from google.colab import drive
drive.mount('/content/gdrive')
!unzip -q "/content/gdrive/My Drive/hvc_data.zip"
# look for `hvc_annotations.csv` file and `resized` dir
%ls 

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive
gdrive/  hvc_annotations.csv  resized/  sample_data/


In [2]:
%tensorflow_version 1.x

import cv2
import json

import numpy as np
import pandas as pd

from functools import partial
from pathlib import Path 
from tqdm import tqdm

from google.colab.patches import cv2_imshow

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, OneHotEncoder


from keras.applications import VGG16
from keras.layers.core import Dropout
from keras.layers.core import Flatten
from keras.layers.core import Dense
from keras.layers import Input
from keras.models import Model
from keras.optimizers import SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator


Using TensorFlow backend.


In [3]:
# load annotations
df = pd.read_csv("hvc_annotations.csv")
del df["filename"] # remove unwanted column
df.head()

,gender,imagequality,age,weight,carryingbag,footwear,emotion,bodypose,image_path
0,male,Average,35-45,normal-healthy,Grocery/Home/Plastic Bag,Normal,Neutral,Front-Frontish,resized/1.jpg
1,female,Average,35-45,over-weight,None,Normal,Angry/Serious,Front-Frontish,resized/2.jpg
2,male,Good,45-55,normal-healthy,Grocery/Home/Plastic Bag,CantSee,Neutral,Front-Frontish,resized/3.jpg
3,male,Good,45-55,normal-healthy,Daily/Office/Work Bag,Normal,Neutral,Front-Frontish,resized/4.jpg
4,female,Good,35-45,slightly-overweight,None,CantSee,Neutral,Front-Frontish,resized/5.jpg


In [4]:
# one hot encoding of labels

one_hot_df = pd.concat([
    df[["image_path"]],
    pd.get_dummies(df.gender, prefix="gender"),
    pd.get_dummies(df.imagequality, prefix="imagequality"),
    pd.get_dummies(df.age, prefix="age"),
    pd.get_dummies(df.weight, prefix="weight"),
    pd.get_dummies(df.carryingbag, prefix="carryingbag"),
    pd.get_dummies(df.footwear, prefix="footwear"),
    pd.get_dummies(df.emotion, prefix="emotion"),
    pd.get_dummies(df.bodypose, prefix="bodypose"),
], axis = 1)

one_hot_df.head().T

,0,1,2,3,4
image_path,resized/1.jpg,resized/2.jpg,resized/3.jpg,resized/4.jpg,resized/5.jpg
gender_female,0,1,0,0,1
gender_male,1,0,1,1,0
imagequality_Average,1,1,0,0,0
imagequality_Bad,0,0,0,0,0
imagequality_Good,0,0,1,1,1
age_15-25,0,0,0,0,0
age_25-35,0,0,0,0,0
age_35-45,1,1,0,0,1
age_45-55,0,0,1,1,0


In [0]:
import keras
import numpy as np

# Label columns per attribute
_gender_cols_ = [col for col in one_hot_df.columns if col.startswith("gender")]
_imagequality_cols_ = [col for col in one_hot_df.columns if col.startswith("imagequality")]
_age_cols_ = [col for col in one_hot_df.columns if col.startswith("age")]
_weight_cols_ = [col for col in one_hot_df.columns if col.startswith("weight")]
_carryingbag_cols_ = [col for col in one_hot_df.columns if col.startswith("carryingbag")]
_footwear_cols_ = [col for col in one_hot_df.columns if col.startswith("footwear")]
_emotion_cols_ = [col for col in one_hot_df.columns if col.startswith("emotion")]
_bodypose_cols_ = [col for col in one_hot_df.columns if col.startswith("bodypose")]

class PersonDataGenerator(keras.utils.Sequence):
    """Ground truth data generator"""

    
    def __init__(self, df, batch_size=16, shuffle=True,augmentation=None):
        self.df = df
        self.batch_size=batch_size
        self.shuffle = shuffle
        self.on_epoch_end()
        self.augmentation = augmentation

    def __len__(self):
        return int(np.floor(self.df.shape[0] / self.batch_size))

    def __getitem__(self, index):
        """fetch batched images and targets"""
        batch_slice = slice(index * self.batch_size, (index + 1) * self.batch_size)
        items = self.df.iloc[batch_slice]
        image = np.stack([cv2.imread(item["image_path"]) for _, item in items.iterrows()])
        target = {
            "gender_output": items[_gender_cols_].values,
            "image_quality_output": items[_imagequality_cols_].values,
            "age_output": items[_age_cols_].values,
            "weight_output": items[_weight_cols_].values,
            "bag_output": items[_carryingbag_cols_].values,
            "pose_output": items[_bodypose_cols_].values,
            "footwear_output": items[_footwear_cols_].values,
            "emotion_output": items[_emotion_cols_].values,
        }
        return image, target

    def on_epoch_end(self):
        """Updates indexes after each epoch"""
        if self.shuffle == True:
            self.df = self.df.sample(frac=1).reset_index(drop=True)


In [8]:
from sklearn.model_selection import train_test_split
train_df, val_df = train_test_split(one_hot_df, test_size=0.20)
train_df.shape, val_df.shape

((10858, 28), (2715, 28))

In [9]:
train_df.head()

,image_path,gender_female,gender_male,imagequality_Average,imagequality_Bad,imagequality_Good,age_15-25,age_25-35,age_35-45,age_45-55,age_55+,weight_normal-healthy,weight_over-weight,weight_slightly-overweight,weight_underweight,carryingbag_Daily/Office/Work Bag,carryingbag_Grocery/Home/Plastic Bag,carryingbag_None,footwear_CantSee,footwear_Fancy,footwear_Normal,emotion_Angry/Serious,emotion_Happy,emotion_Neutral,emotion_Sad,bodypose_Back,bodypose_Front-Frontish,bodypose_Side
10038,resized/10039.jpg,1,0,0,0,1,1,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,0,1,0
11196,resized/11198.jpg,0,1,1,0,0,0,0,0,0,1,0,1,0,0,1,0,0,0,0,1,1,0,0,0,0,1,0
8431,resized/8432.jpg,1,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,1,0,1,0,0
6327,resized/6328.jpg,1,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,1,0,1,0,0,0,1,0
5020,resized/5021.jpg,1,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0


In [0]:
# create train and validation data generators
train_gen = PersonDataGenerator(train_df, batch_size=16, 
                                augmentation=ImageDataGenerator(rescale=1./255,
                                                                featurewise_center=True,
                                                                featurewise_std_normalization=True,
                                                                horizontal_flip=True,vertical_flip=False,rotation_range=20))
valid_gen = PersonDataGenerator(val_df, batch_size=16, shuffle=False,
                                augmentation=ImageDataGenerator(rescale=1./255,
                                                                featurewise_center=True,
                                                                featurewise_std_normalization=True))

In [15]:
# get number of output units from data
images, targets = next(iter(train_gen))
# targets is dictionary data from person data generator.
num_units = { k.split("_output")[0]:v.shape[1] for k, v in targets.items()}
num_units

{'age': 5,
 'bag': 3,
 'emotion': 4,
 'footwear': 3,
 'gender': 2,
 'image_quality': 3,
 'pose': 3,
 'weight': 4}

In [19]:
from keras.applications.resnet50 import ResNet50

backbone =ResNet50(
    weights="imagenet", 
    include_top=False, 
    input_tensor=Input(shape=(224, 224, 3))
)

neck = backbone.output  #backbone.output gives the last layer already build by ResNet50 on the above statement 
neck = Flatten(name="flattener")(neck)  # Adds Flatten layer
neck = Dense(1014, activation="relu")(neck) # Adds fully connected layer with Relu Activation

def build_tower(in_layer):
    neck = Dropout(0.2)(in_layer)
    neck = Dense(256, activation="relu")(neck)
    neck = Dropout(0.3)(in_layer)
    neck = Dense(256, activation="relu")(neck)
    return neck


def build_head(name, activator,in_layer):
    return Dense(
        num_units[name], activation=activator, name=f"{name}_output"
    )(in_layer)

# heads
gender = build_head("gender", "sigmoid", build_tower(neck))  #gender is binary classification
image_quality = build_head("image_quality", "sigmoid",build_tower(neck)) # image is multi-class single label, can be either good, avg or bad
age = build_head("age", "sigmoid",build_tower(neck))  # Age is regression to arbitrary value but here we are didving to groups like 15-25, 25-35, hence multi-class single label
weight = build_head("weight", "sigmoid",build_tower(neck)) # weight is multi-class, single label as it can healthy, over or under
bag = build_head("bag", "sigmoid",build_tower(neck))  # bag is multi-class, single label 
footwear = build_head("footwear","sigmoid", build_tower(neck))  # multi-class, single label 
emotion = build_head("emotion", "sigmoid", build_tower(neck))  # multi-class, single label 
pose = build_head("pose", "sigmoid",build_tower(neck)) # multi-class, single label


model = Model(
    inputs=backbone.input, 
    outputs=[gender, image_quality, age, weight, bag, footwear, pose, emotion]
)

/usr/local/lib/python3.6/dist-packages/keras_applications/resnet50.py:265: UserWarning: The output shape of `ResNet50(include_top=False)` has been changed since Keras 2.2.0.
  warnings.warn('The output shape of `ResNet50(include_top=False)` '


In [0]:
# freeze backbone
for layer in backbone.layers:
	layer.trainable = False

In [22]:
opt = SGD(lr=0.001, momentum=0.9)
model.compile(
    optimizer=opt,
    loss="categorical_crossentropy",  
    metrics=["accuracy"]
)

In [23]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=6, 
    epochs=10,
    verbose=1
)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


Epoch 1/10
678/678 [==============================] - 70s 103ms/step - loss: 9.4762 - gender_output_loss: 0.6956 - image_quality_output_loss: 1.0998 - age_output_loss: 1.6116 - weight_output_loss: 1.3880 - bag_output_loss: 1.0970 - footwear_output_loss: 1.0999 - pose_output_loss: 1.1013 - emotion_output_loss: 1.3829 - gender_output_acc: 0.4439 - image_quality_output_acc: 0.4994 - age_output_acc: 0.2307 - weight_output_acc: 0.6178 - bag_output_acc: 0.3068 - footwear_output_acc: 0.3675 - pose_output_acc: 0.1663 - emotion_output_acc: 0.1233 - val_loss: 9.4696 - val_gender_output_loss: 0.6931 - val_image_quality_output_loss: 1.0986 - val_age_output_loss: 1.6094 - val_weight_output_loss: 1.3863 - val_bag_output_loss: 1.0986 - val_footwear_output_loss: 1.0986 - val_pose_output_loss: 1.0986 - val_emotion_output_loss: 1.3863 - val_gender_output_acc: 0.4271 - val_image_quality_output_acc: 0.5551 - va

In [24]:
model.fit_generator(
    generator=train_gen,
    validation_data=valid_gen,
    use_multiprocessing=True,
    workers=10, 
    epochs=20,
    verbose=1
)

Epoch 1/20
678/678 [==============================] - 58s 85ms/step - loss: 9.4590 - gender_output_loss: 0.6931 - image_quality_output_loss: 1.0986 - age_output_loss: 1.6094 - weight_output_loss: 1.3863 - bag_output_loss: 1.0986 - footwear_output_loss: 1.0880 - pose_output_loss: 1.0986 - emotion_output_loss: 1.3863 - gender_output_acc: 0.4402 - image_quality_output_acc: 0.5450 - age_output_acc: 0.1847 - weight_output_acc: 0.6251 - bag_output_acc: 0.3414 - footwear_output_acc: 0.3698 - pose_output_acc: 0.1621 - emotion_output_acc: 0.1100 - val_loss: 9.4054 - val_gender_output_loss: 0.6931 - val_image_quality_output_loss: 1.0986 - val_age_output_loss: 1.6094 - val_weight_output_loss: 1.3863 - val_bag_output_loss: 1.0986 - val_footwear_output_loss: 1.0344 - val_pose_output_loss: 1.0986 - val_emotion_output_loss: 1.3863 - val_gender_output_acc: 0.4275 - val_image_quality_output_acc: 0.5562 - val_age_output_acc: 0.1779 - val_weight_output_acc: 0.6250 - val_bag_output_acc: 0.3280 - val_footw